# Step 0: Getting everything ready

 We need to create the right folders

In [ ]:
import os
import cv2
general_videos_folder = "./videos/"
inputs_subfolder = "inputs/"
audio_subfolder = "audios/"
frames_subfolder = "frames/"
aligned_frames_subfolder = "aligned_frames/"
processed_frames_subfolder = "processed_frames/"
no_sound_videos_subfolder = "no_sound_videos/"
final_videos_subfolder = "final_videos/"
os.makedirs(general_videos_folder, exist_ok = True)
os.makedirs(f"{general_videos_folder}{inputs_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{audio_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{processed_frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{no_sound_videos_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{final_videos_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{aligned_frames_subfolder}", exist_ok = True)

This will be the name of the video we are trying to process

In [ ]:
filename = "news_base"

# Step 1: Break the video into multiple frames

For this we use the cv2 library. This code is based on the [following link](https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/)

In [ ]:
def video_to_frame(filename):
  os.makedirs(f"{general_videos_folder}{frames_subfolder}{filename}/", exist_ok = True)
  get_frame_path = lambda part: f"{general_videos_folder}{frames_subfolder}{filename}/frame-{part}.jpg" 
  vid_obj = cv2.VideoCapture(f"{general_videos_folder}{inputs_subfolder}{filename}.mp4")
  count = 0
  print(f"Total # of frames according to code {int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))}")
  fps = vid_obj.get(cv2.CAP_PROP_FPS)
  print(f"Frames per second = {fps}")
  print("Now reading the frames... [This might take a while]")
  while True: 
    success, image = vid_obj.read() 
    if not success:
      break
    # Saves the frames with frame-count 
    cv2.imwrite(get_frame_path(count), image) 
    count += 1
  print(f"Total # frames accroding to loop = {count}")
  print(f"Succesfully wrote all the frames in the folder {general_videos_folder}{frames_subfolder}{filename}/")
  return fps

Apply function to our video

In [ ]:
video_fps = video_to_frame(filename) #this function return the fps of such video
print(f"FPS returned from function: {video_fps}")

# Step 2: Updating every frame

The code from this repo assumes you are using **tensorflow-gpu:1.14.0**, so let's uninstall any previous tensorflow instance you may have

In [ ]:
!pip uninstall tensorflow

Now let's install the necessary tensorflow

In [ ]:
!pip install tensorflow-gpu==1.14

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Making sure that the tensorflow is the correct (1.14.0)

In [ ]:
print(tf.__version__) #Making sure the tensorflow version is correct

Importing necessary libraries

In [ ]:
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
import matplotlib.pyplot as plt

In [ ]:
#URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
#URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
URL_FFHQ = 'https://drive.google.com/uc?id=1PrjiBIXQ-tEa3l3vAtBR7FLi-n9WnVB8'
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)
generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

**Define some useful functions**

In [ ]:
def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((256, 256))

def move_and_show(latent_vector, direction, coeffs):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ax[i].imshow(generate_image(new_latent_vector))
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

def save_modified_image(latent_vector, direction, coeff, output):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
    img = generate_image(new_latent_vector)
    img = img.save(output) 

### Now we are first getting the aligned images out of every frame

In [ ]:
os.environ['FRAMES_FOLDER'] =  f"{general_videos_folder}{frames_subfolder}{filename}"
os.environ['ALIGNED_FOLDER'] = f"{general_videos_folder}{aligned_frames_subfolder}{filename}"
#need to make sure that the subfolder {filename}/ exists

In [ ]:
!python align_images.py $FRAMES_FOLDER $ALIGNED_FOLDER  